In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

data = pd.read_csv("https://raw.githubusercontent.com/Gaelim/google-data-stuido/main/Ecommerce_Data.csv").drop(["Unnamed: 0"],axis=1)
data["Date"] = pd.to_datetime(data["Date"])
data["CustomerID"] = data["CustomerID"].astype(int)
data = data[data['Quantity']>0]

In [2]:
data["Total_Revenue"] = data["Quantity"] * data["UnitPrice"]

In [3]:
data

,InvoiceNo,StockCode,Description,Quantity,UnitPrice,CustomerID,Country,Date,Hour,Total_Revenue
0,574477,22591,CARDHOLDER GINGHAM CHRISTMAS TREE,1,3.25,15453,United Kingdom,2011-11-04,12,3.25
1,570275,23541,WALL ART CLASSIC PUDDINGS,12,7.45,13098,United Kingdom,2011-10-10,10,89.40
2,566482,22508,DOORSTOP RETROSPOT HEART,12,3.75,16609,United Kingdom,2011-09-13,9,45.00
3,541215,22662,LUNCH BAG DOLLY GIRL DESIGN,10,1.65,14329,United Kingdom,2011-01-14,13,16.50
4,565930,POST,POSTAGE,5,18.00,12685,France,2011-09-08,10,90.00
...,...,...,...,...,...,...,...,...,...,...
81596,560692,21294,ETCHED GLASS COASTER,4,0.39,14704,United Kingdom,2011-07-20,12,1.56
81597,540187,22569,FELTCRAFT CUSHION BUTTERFLY,4,3.75,15358,United Kingdom,2011-01-05,13,15.00
81598,547871,17012C,ORIGAMI LAVENDER INCENSE/CANDL SET,5,2.55,16931,United Kingdom,2011-03-27,14,12.75
81599,562129,22138,BAKING SET 9 PIECE RETROSPOT,3,4.95,13186,United Kingdom,2011-08-03,9,14.85


In [4]:
max_date = data["Date"].max()
data_new = data.groupby('CustomerID').agg({
    'Date' : lambda x: (max_date - x.min()).days,
    'InvoiceNo': lambda x: len(x),
    'Quantity' : lambda x: x.sum(),
    'Total_Revenue' : lambda x: x.sum()
})
data_new.columns = ['Age', 'Num_Transactions', 'Quantity', 'Total_Revenue']

In [5]:
data_new = data_new[data_new['Quantity']>0]
data_new

,Age,Num_Transactions,Quantity,Total_Revenue
CustomerID,,,,
12347,367,48,623,1146.59
12348,358,7,531,550.92
12349,18,16,203,406.32
12350,310,2,36,45.60
12352,296,21,163,663.33
...,...,...,...,...
18280,277,3,9,52.75
18281,180,2,11,33.45
18282,126,4,11,67.85


In [6]:
data_new['AOV'] = data_new['Total_Revenue'] / data_new['Num_Transactions']

In [7]:
# Purchase Frequency
purc_freq = sum(data_new['Num_Transactions']) / len(data_new)
purc_freq

19.496336101612115

In [8]:
# Repeate Rate
Repeate_Rate = data_new[data_new['Num_Transactions']>1].shape[0] / data_new.shape[0]
Repeate_Rate

0.9291646311675623

In [9]:
# Churn Rate
churn_rate = 1-Repeate_Rate
churn_rate

0.07083536883243768

In [10]:
# Profit Margin
data_new["Profit Margin"] = data_new['Total_Revenue']*.10
data_new

,Age,Num_Transactions,Quantity,Total_Revenue,AOV,Profit Margin
CustomerID,,,,,,
12347,367,48,623,1146.59,23.887292,114.659
12348,358,7,531,550.92,78.702857,55.092
12349,18,16,203,406.32,25.395000,40.632
12350,310,2,36,45.60,22.800000,4.560
12352,296,21,163,663.33,31.587143,66.333
...,...,...,...,...,...,...
18280,277,3,9,52.75,17.583333,5.275
18281,180,2,11,33.45,16.725000,3.345
18282,126,4,11,67.85,16.962500,6.785


In [11]:
data_new["CLTV"] = (data_new['AOV']*purc_freq/churn_rate)*.10

In [12]:
data_new.sort_values('CLTV',ascending=False)

,Age,Num_Transactions,Quantity,Total_Revenue,AOV,Profit Margin,CLTV
CustomerID,,,,,,,
15749,332,4,8898,20713.30,5178.325000,2071.330,142525.360293
16000,2,5,2630,7948.30,1589.660000,794.830,43752.924786
17949,289,10,4565,7671.45,767.145000,767.145,21114.475728
16333,298,11,7840,7944.40,722.218182,794.440,19877.934771
12931,239,15,5590,9344.82,622.988000,934.482,17146.777994
...,...,...,...,...,...,...,...
17859,304,5,5,5.21,1.042000,0.521,28.679433
17962,142,3,3,2.99,0.996667,0.299,27.431703
14661,26,1,1,0.85,0.850000,0.085,23.394931
